In [1]:
!pip install --upgrade --user google-cloud-aiplatform google-cloud-bigquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.3/237.3 kB 29.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
# Get project ID
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-west1" # @param {type:"string"}
print(PROJECT_ID)

qwiklabs-gcp-03-db829e6a1f04


In [2]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

print("Initialized")

Initialized


In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

def answer_question_gemini(prompt):
  model = GenerativeModel("gemini-pro")
  response = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": 8192,
        "temperature": 0.5,
        "top_p": 0.5,
        "top_k": 10,
    },
  stream=False,
  )
  try:
    return response.text
  except:
    print("An Error Ocuured Cleaning the Data")
    return "An Error Ocuured Cleaning the Data"

In [4]:
def run_search(question):
  from google.cloud import bigquery

  client = bigquery.Client()

  sql = """
      SELECT query.query, base.title, base.body
      FROM VECTOR_SEARCH(
      TABLE `bbc_news.bbc_news_with_embeddings_table`, 'text_embedding',
      (
      SELECT text_embedding, content AS query
      FROM ML.GENERATE_TEXT_EMBEDDING(MODEL `bbc_news.embedding_model`,
          (SELECT @question AS content))),
      top_k => 5)
      """

  job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("question", "STRING", question),
    ]
  )

  query_job = client.query(sql, job_config=job_config)

  data = ""
  for row in query_job:
    data += row.body + "\n"

  return data

In [5]:
def build_prompt(data, question):
  prompt = """
    Instructions: Answer the question using the following Context.

    Context: {0}

    Question: {1}
  """.format(data, question)
  return prompt

In [6]:
from IPython.core.display import display, HTML

def answer_question(question):

  data = run_search(question)
  display("Retrieved Data:")
  display(data)
  display(" . . . ")
  prompt = build_prompt(data, question)
  answer_gemini = answer_question_gemini(prompt)

  return answer_gemini

In [7]:
QUESTION = "Tell me about the US Economy"

answer_gemini = answer_question(QUESTION)
display("User Question:")
display(QUESTION)
display("--------------------------------")
display("Gemini Answer:")
display(answer_gemini)

'Retrieved Data:'

'Most areas of the US saw their economy continue to expand in December and early January, the US Federal Reserve said in its latest Beige Book report.\n\nOf the 12 US regions it identifies for the study, 11 showed stronger economic growth, with only the Cleveland area falling behind with a "mixed" rating. Consumer spending was higher in December than November, and festive sales were also up on 2003. The employment picture also improved, the Fed said.\n\n"Labour markets firmed in a number of districts, but wage pressures generally remained modest," the Beige Book said. "Several districts reported higher prices for building materials and manufacturing inputs, but most reported steady or only slightly higher overall price levels." The report added that residential real estate activity remained strong and that commercial real estate activity strengthened in most districts. "Office leasing was especially brisk in Washington DC, and New York City, two of the nation\'s strongest commercial ma

' . . . '

'User Question:'

'Tell me about the US Economy'

'--------------------------------'

'Gemini Answer:'

"## US Economy in 2004: A Summary\n\nThe US economy experienced robust growth in 2004, exceeding expectations with a 4.4% annual growth rate. This positive performance was driven by several factors:\n\n* **Strong consumer spending:** Consumer spending remained high throughout the year, fueled by strong employment and rising wages.\n* **Increased business investment:** Businesses significantly increased their spending on capital equipment and software, contributing to the overall economic expansion.\n* **Strong residential and commercial real estate activity:** Both residential and commercial real estate markets remained robust, with strong demand and rising prices.\n* **Improved employment picture:** The labor market saw continued improvement, with more jobs created and modest wage growth.\n\nHowever, some concerns loomed on the horizon:\n\n* **Potential interest rate hikes:** The strong economic performance could lead to interest rate increases, potentially slowing down the economy.\n

In [8]:
QUESTION = "What it a good recipe for bouillabaisse"

answer_gemini = answer_question(QUESTION)
display("User Question:")
display(QUESTION)
display("--------------------------------")
display("Gemini Answer:")
display(answer_gemini)

'Retrieved Data:'

'Iraq is to invite bids for two telephone licences, saying it wants to significantly boost nationwide coverage over the next decade.\n\nBids have been invited from local, Arab and foreign companies, Iraq\'s Ministry of Communications said. The winner will work in partnership with the Iraqi Telecommunications and Post Company (ITPC). The firms will install and operate a fixed phone network, providing voice, fax and internet services.\n\nThe ministry said that it wanted to increase Iraq\'s "very low telephone service penetration rate from about 4.5% today to about 25% within 10 years." It also hopes to develop a "highly visible and changeable telecommunication sector". Details of the bidding and tender process will be published on the ministry\'s website on 9 February. It also is planning a road-show for investors in Amman, Jordan. The ministry said it would base its selection on criteria including the speed of implementation, tariff rates, coverage, and the firm\'s experience and financ

' . . . '

'User Question:'

'What it a good recipe for bouillabaisse'

'--------------------------------'

'Gemini Answer:'

"I am sorry, but the context provided does not contain any information about a recipe for bouillabaisse. The context is about Iraq inviting bids for telephone licenses, Italy's rugby team, and a group of MPs visiting Scotland."